
### <div style="text-align: justify;">EXEMPLO DO LIVRO: PESQUISA OPERACIONAL PARA OS CURSOS DE ADMINISTRAÇÃO E ENGENHARIA, 4ª EDIÇÃO.</div>
#### AUTORES: ERMES MEDEIROS DA SILVA, ELIO MEDEIROS DA SILVA, VALTER GONÇALVES E AFRÂNIO CARLOS MUROLO.




**OBSERVAÇÃO:**

<div style="text-align: justify;">A base do exemplo de simulação abaixo foi retirada do livro citado, porém tive que adaptar os dados, pois não foi possível saber ao certo os dados de entrada (demanda semanal). Portanto, neste notebook, mostro a ideia do exemplo do livro com valores diferentes de entrada. O exemplo retirado do livro citado será replicado por mim com a linguagem R.</div>

In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)
suppressPackageStartupMessages({library(dplyr)})

### SIMULAÇÃO DE MONTE CARLO
    
#### EXEMPLO DE APLICAÇÃO
<p>
<div style="text-align: justify;">Um feirante faz compra de ovos uma vez por semana num entreposto atacadista. Os ovos não vendidos dentro de uma semana se estragam, e são descartados, acarretando prejuízo de 400 u.m por dúzia. Por outro lado, a falta de produto para a venda também acarreta perda, estimada em 150 u.m por dúzia demandada e não vendida. O feirante anotou a demanda das últimas 40 semanas e dividiu-as em sete classes, conforme abaixo:</div>

#### DADOS

In [2]:
# DADOS DA DEMANDA SEMANAL DE DÚZIAS
dados_semanais <- c(210,216,222,216,210,228,234,240,234,216,222,216,204,210,222,228,240,234,216,228,
                    252,248,252,256,252,248,234,240,234,248,216,228,222,234,228,222,234,240,234,228)

#### TESTAR AS HIPÓTESES

1. Comprar cada semana a demanda efetiva da semana anterior.
2. Comprar uma quantidade igual à média histórica anotada no período anterior de 40 semanas (média = valor inteiro mais próximo da média verificada).
3. O exame dos resultados sugere o teste de outra hipótese.

In [3]:
summary(dados_semanais)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  204.0   220.5   228.0   229.9   240.0   256.0 

In [4]:
n <- length(dados_semanais); cat("Tamanho(n):",n)
R <- max(dados_semanais) - min(dados_semanais); cat("\nAplitude(R):",R)
k <- round(sqrt(n),2); cat("\nClasses(k):",k)
h <- round(R/k, 2); cat("\nIntervalo de classe(h):",h)

Tamanho(n): 40
Aplitude(R): 52
Classes(k): 6.32
Intervalo de classe(h): 8.23

#### TABELA DE FREQUÊNCIA

In [5]:
# INTERVALOS
intervalos <- seq(204, 260, 6); cat("Intervalos:", intervalos)
# CLASSES
classes <- cut(x = dados_semanais, breaks = intervalos, right = FALSE, include.lowest = TRUE)
# FREQUÊNCIA 
frequencia <- table(classes)
frequencia_df <- data.frame(frequencia)
# TABELA DE FREQUÊNCIA
tabela_frequencia <- frequencia_df %>%
mutate(freq_R.1000 = Freq/n * 1000,
       freq_RA = cumsum(Freq)/n * 1000)

Intervalos: 204 210 216 222 228 234 240 246 252 258

In [6]:
# CRIAR VETOR COM O PONTO MÉDIO DAS CLASSES DA TABELA DE FREQUÊNCIA
PM = c()
num = 0; soma = 1
for(x in 1:length(frequencia_df$classes)){
    num = num + soma
    ponto_medio <- (intervalos[num] + intervalos[num+1])/2
    PM <- append(PM, ponto_medio)
}     

In [7]:
# ADICIONAR O VETOR À TABELA DE FREQUÊNCIAS
tabela_frequencia <- cbind(tabela_frequencia, PM)
# COLOCAR AS COLUNAS DA TABELA NA ORDEM DESEJADA
tabela_frequencia <- select(tabela_frequencia, classes, PM, everything())

<div style="text-align: justify;">Vamos simular a primeira hipótese para 20 semanas. Os limites para os números aleatórios são obtidos através da frequência acumulada relativa, conforme mostra a tabela.</div>

In [8]:
# CRIAR LIMITES INFERIORES E SUPERIORES PARA SORTEIO
tabela_frequencia <- tabela_frequencia %>%
mutate(LI_sorteio = freq_RA - freq_R.1000,
       LS_sorteio = freq_RA - 1); tabela_frequencia

classes,PM,Freq,Freq.R.1000,Freq.R.A,L.I.sorteio,L.S.sorteio
"[204,210)",207,1,25,25,0,24
"[210,216)",213,3,75,100,25,99
"[216,222)",219,6,150,250,100,249
"[222,228)",225,5,125,375,250,374
"[228,234)",231,6,150,525,375,524
"[234,240)",237,8,200,725,525,724
"[240,246)",243,4,100,825,725,824
"[246,252)",249,3,75,900,825,899
"[252,258]",255,4,100,1000,900,999


In [9]:
# FUNÇÕES ESPECÍFICAS PARA ESTE PROBLEMA DE NEGÓCIO
# CRIAR A FUNÇÃO VENDA - (DEMANDA E ESTOQUE INICIAL)
funcao_venda = function(x,y){
    if(x < y)
        x 
    else
        y 
}
# CRIAR A FUNÇÃO DE ESTOQUE FINAL - (ESTOQUE INICIAL - VENDA)
funcao_estoque_final = function(x,y){
        z = x - y
}
# CRIAR A FUNÇÃO CUSTO DE FALTA - (DEMANDA - ESTOQUE INICIAL)*150.00
funcao_custo_falta = function(x,y){
    if(x > y)
        x = (x - y) * 150.00
    else
        y = 0
}
# CRIAR FUNÇÃO CUSTO DE SOBRA - (ESTOQUE FINAL*400)
funcao_custo_sobra = function(x){
        y = x * 400
}
# CRIAR A FUNÇÃO CUSTO TOTAL - (CUSTO FALTA + CUSTO SOBRA)
funcao_custo_total = function(x, y){
        z = x + y
}

#### FUNÇÃO SIMULAÇÃO DE MONTE CARLO

In [10]:
# FUNÇÃO SIMULAÇÃO DE MONTE CARLO
SimulacaoMonteCarlo <- function(iteracoes,coluna_LI,coluna_LS,coluna_PM){
    simulacao <- data.frame()
    for (i in 1:iteracoes){
        iteracao <- i
        sorteio <- sample(0:999, 1)
        contador = 0
        repeat {
        contador = contador + 1
        if (sorteio >= coluna_LI[contador] & sorteio <= coluna_LS[contador]){
            PM_classe <- coluna_PM[contador]
            vetor = c(iteracao,sorteio,PM_classe)
            simulacao = rbind(simulacao, vetor)
            colnames(simulacao) <- c("iteracoes","sorteio","previsao")
            break()
        
    }
    } 
}
    return(simulacao)
}

#### TESTANDO A PRIMEIRA HIPÓTESE
1. Comprar cada semana a demanda efetiva da semana anterior.

#### CRIANDO A TABELA DE CUSTOS
<p>
<div style="text-align: justify;">A partir da tabela de frequência com os números aleatórios, levantamos uma sequência de vinte números de três algarismos, como já anteriormente descrito. Para iniciar o processo, vamos aceitar a demanda anterior à primeira semana igual à média ponderada do período anterior.</div>

In [11]:
# COMPRAR CADA SEMANA A DEMANDA EFETIVA DA SEMANA ANTERIOR - ESTOQUE INICIAL
# MÉDIA PONDERADA DOS DADOS FOI COLOCADA NO ESTOQUE INICIAL DA PRIMEIRA SEMANA, POIS NÃO SE TEM A DEMANDA ANTERIOR, JÁ QUE 
# A SIMULAÇÃO COMEÇA NA PRIMEIRA ITERAÇÃO.
media_ponderada <- round(weighted.mean(tabela_frequencia$PM, tabela_frequencia$Freq))
estoque_inicial <- c(media_ponderada); estoque_inicial

[1] 233

In [12]:
tabela_custos <- SimulacaoMonteCarlo(20, tabela_frequencia$LI_sorteio, tabela_frequencia$LS_sorteio, tabela_frequencia$PM)
colnames(tabela_custos) <- c("semana", "sorteio", "demanda")

In [13]:
# CRIAR A COLUNA ESTOQUE INICIAL 
# A PARTIR DAS PREVISÕES DE DEMANDA DA PRIMEIRA SEMANA COLOCA-SE NO ESTOQUE INICIAL A DEMANDA DA SEMANA ANTERIOR.
tabela_custos$estoque_inicial[1] <- estoque_inicial
tabela_custos$estoque_inicial[2:20] <- tabela_custos$demanda[1:19]

In [14]:
# CRIAR A COLUNA VENDA
tabela_custos$venda <- mapply(funcao_venda, tabela_custos$demanda, tabela_custos$estoque_inicial)
# CRIAR A COLUNA ESTOQUE FINAL
tabela_custos$estoque_final <- mapply(funcao_estoque_final, tabela_custos$estoque_inicial, tabela_custos$venda)
# CRIAR A COLUNA CUSTO DE FALTA
tabela_custos$custo_falta <- mapply(funcao_custo_falta, 
                                          tabela_custos$demanda, tabela_custos$estoque_inicial)
# CRIAR COLUNA CUSTO SOBRA 
tabela_custos$custo_sobra <- funcao_custo_sobra(tabela_custos$estoque_final)
# CRIAR A COLUNA CUSTO TOTAL
tabela_custos$custo_total <- funcao_custo_total(tabela_custos$custo_falta, tabela_custos$custo_sobra)

In [15]:
tabela_custos

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
1,426,231,233,231,2,0,800,800
2,513,231,231,231,0,0,0,0
3,887,249,231,231,0,2700,0,2700
4,506,231,249,231,18,0,7200,7200
5,614,237,231,231,0,900,0,900
6,426,231,237,231,6,0,2400,2400
7,235,219,231,219,12,0,4800,4800
8,715,237,219,219,0,2700,0,2700
9,198,219,237,219,18,0,7200,7200
10,888,249,219,219,0,4500,0,4500


In [16]:
custo_total <- sum(tabela_custos$custo_total)
cat("Custo Total:", custo_total,"u.m")

Custo Total: 74300 u.m

#### TESTANDO A SEGUNDA HIPÓTESE
2. Comprar uma quantidade igual à média histórica anotada no período anterior de 40 semanas (média = valor inteiro mais próximo da média verificada).

In [17]:
cat("Média Ponderada =" ,media_ponderada, "\nComprar", media_ponderada,"toda semana")

Média Ponderada = 233 
Comprar 233 toda semana

In [18]:
# CRIAR A TABELA DE CUSTO DA SEGUNDA HIPÓTESE COM BASE NA PRIMEIRA TABELA GERADA
tabela_custos2 = tabela_custos

In [19]:
# CRIAR A COLUNA ESTOQUE INICIAL COM A MÉDIA PONDERADA
tabela_custos2$estoque_inicial[1:20] <- estoque_inicial

In [20]:
# CRIAR A COLUNA VENDA
tabela_custos2$venda <- mapply(funcao_venda, tabela_custos2$demanda, tabela_custos2$estoque_inicial)
# CRIAR A COLUNA ESTOQUE FINAL
tabela_custos2$estoque_final <- mapply(funcao_estoque_final, tabela_custos2$estoque_inicial, tabela_custos2$venda)
# CRIAR A COLUNA CUSTO DE FALTA
Tabela_custos2$custo_falta <- mapply(funcao_custo_falta, 
                                          tabela_custos2$demanda, tabela_custos2$estoque_inicial)
# CRIAR COLUNA CUSTO SOBRA 
tabela_custos2$custo_sobra <- funcao_custo_sobra(tabela_custos2$estoque_inicial)
# CRIAR A COLUNA CUSTO TOTAL
tabela_custos2$custo_total <- funcao_custo_total(tabela_custos2$custo_falta, tabela_custos2$custo_sobra)

In [21]:
tabela_custos2

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
1,426,231,233,231,2,0,800,800
2,513,231,233,231,2,0,800,800
3,887,249,233,233,0,2400,0,2400
4,506,231,233,231,2,0,800,800
5,614,237,233,233,0,600,0,600
6,426,231,233,231,2,0,800,800
7,235,219,233,219,14,0,5600,5600
8,715,237,233,233,0,600,0,600
9,198,219,233,219,14,0,5600,5600
10,888,249,233,233,0,2400,0,2400


In [22]:
custo_total2 <- sum(tabela_custos2$custo_total)
cat("Custo Total:", custo_total2,"u.m")

Custo Total: 41300 u.m

<div style="text-align: justify;">O segundo quadro foi, até agora, o mais favorável. Observando a distribuição dos custos parece razoável pensar na hipótese de uma compra menor que a média histórica.</div>

#### TESTANDO A TERCEIRA HIPÓTESE
3. O exame dos resultados sugere o teste de outra hipótese.

In [23]:
cat("Compra de 230 dúzias por semana")

Compra de 230 dúzias por semana

In [24]:
# CRIAR A TABELA DE CUSTO DA TERCEIRA HIPÓTESE COM BASE NA PRIMEIRA TABELA GERADA
tabela_custos3 = tabela_custos

In [25]:
# CRIAR A COLUNA ESTOQUE INICIAL COM A QUANTIDADE DE 230 DÚZIAS
tabela_custos3$estoque_inicial[1:20] <- 230

In [26]:
# CRIAR A COLUNA VENDA
tabela_custos3$venda <- mapply(funcao_venda, tabela_custos3$demanda, tabela_custos3$estoque_inicial)
# CRIAR A COLUNA ESTOQUE FINAL
tabela_custos3$estoque_final <- mapply(funcao_estoque_final, tabela_custos3$estoque_inicial, tabela_custos3$venda)
# CRIAR A COLUNA CUSTO DE FALTA
tabela_custos3$custo_falta <- mapply(funcao_custo_falta, 
                                          tabela_custos3$demanda, tabela_custos3$estoque_inicial)
# CRIAR COLUNA CUSTO SOBRA 
tabela_custos3$custo_sobra <- funcao_custo_sobra(tabela_custos3$funcao_estoque_final)
# CRIAR A COLUNA CUSTO TOTAL
tabela_custos3$custo_total <- funcao_custo_total(tabela_custos3$custo_falta, tabela_custos3$custo_sobra)

In [27]:
tabela_custos3

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
1,426,231,230,230,0,150,0,150
2,513,231,230,230,0,150,0,150
3,887,249,230,230,0,2850,0,2850
4,506,231,230,230,0,150,0,150
5,614,237,230,230,0,1050,0,1050
6,426,231,230,230,0,150,0,150
7,235,219,230,219,11,0,4400,4400
8,715,237,230,230,0,1050,0,1050
9,198,219,230,219,11,0,4400,4400
10,888,249,230,230,0,2850,0,2850


In [28]:
custo_total3 <- sum(tabela_custos3$custo_total)
cat("Custo Total:", custo_total3,"u.m")

Custo Total: 37100 u.m

Conclusão: das três hipóteses testadas, a terceira parece a mais favorável.